## 02 Data Cleaning and Preprocessing
**Objetivo**: Limpiar y preprocesar los datos para que estén listos para el modelado.
**Contenido**:
- Manejo de valores nulos.
- Creación de nuevas características (feature engineering).
- Conversión de tipos de datos.
- Codificación de variables categóricas.
- Normalización y estandarización de las variables.



In [1]:
## Librerías 
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
import os

##  Cargamos los Dataset 

- Trabajaremos exclusivamente con el dataset de entrenamiento "train_transaction" para obtener tanto los datos de entrenamiento como los de validación. De este dataset solo tomaremos el 10 % del total de los registro.

In [2]:
select_col_transaction = 'TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9'.split(',')
select_col_identity = 'TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo'.split(',')
data_dir = '../data/raw/ieee-fraud-detection'

### Selección de porción del Dataset de entrenamiento con Muestreo Estratificado



In [3]:
## dataset de entrenamiento
seed = 42
# Cargar los datos
df_transaction_train = pd.read_csv(data_dir + '/train_transaction.csv', usecols=select_col_transaction)
df_identity_train = pd.read_csv(data_dir + '/train_identity.csv', usecols=select_col_identity)

# Combinar los datasets
dataset = pd.merge(df_transaction_train, df_identity_train, on='TransactionID', how='left')

# Realizar el muestreo estratificado
#data, _ = train_test_split(dataset, stratify=dataset['isFraud'], test_size=0.8, random_state=seed)
data = dataset
# Mostrar la información del dataset resultante
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 95 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   TransactionID   590540 non-null  int64  
 1   isFraud         590540 non-null  int64  
 2   TransactionDT   590540 non-null  int64  
 3   TransactionAmt  590540 non-null  float64
 4   ProductCD       590540 non-null  object 
 5   card1           590540 non-null  int64  
 6   card2           581607 non-null  float64
 7   card3           588975 non-null  float64
 8   card4           588963 non-null  object 
 9   card5           586281 non-null  float64
 10  card6           588969 non-null  object 
 11  addr1           524834 non-null  float64
 12  addr2           524834 non-null  float64
 13  dist1           238269 non-null  float64
 14  dist2           37627 non-null   float64
 15  P_emaildomain   496084 non-null  object 
 16  R_emaildomain   137291 non-null  object 
 17  C1        

### Armamos un nuevo dataset con las columnas más relevantes

In [4]:
data = data[['isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD','addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
       'DeviceType', 'DeviceInfo']].copy()

## Manejo de valores nulos

- Basándonos en el análisis exploratorio de datos (EDA) realizado previamente, eliminamos las columnas que contienen más del 80% de valores nulos.
- Eliminamos las filas con valores nulos debido a que tenemos suficientes cantidad de registros 

In [5]:
data.drop(['dist1','DeviceType','R_emaildomain','DeviceInfo','dist2'], axis=1,inplace=True)

In [6]:
data.dropna(inplace=True)

In [7]:
data.isnull().sum()

isFraud           0
TransactionDT     0
TransactionAmt    0
ProductCD         0
addr1             0
addr2             0
P_emaildomain     0
dtype: int64

## Conversión de datos 

No es necesatio convertir los tipos de datos, debido a que son adecuado al tipo de datos que ya tienen.

In [8]:
data.dtypes

isFraud             int64
TransactionDT       int64
TransactionAmt    float64
ProductCD          object
addr1             float64
addr2             float64
P_emaildomain      object
dtype: object

## Separamos datos características (X) y variable objetivo (y)

In [9]:
X = data.drop('isFraud', axis=1)  # features
y = data['isFraud']  # target

## Creación de nuevas características (feature engineering).

In [10]:

# Calcular los cuartiles y el IQR
Q1 = np.percentile(X['TransactionAmt'], 25)
Q3 = np.percentile(X['TransactionAmt'], 75)
IQR = Q3 - Q1

# Definir los umbrales para valores atípicos
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Función para clasificar los valores en rangos
def classify_transaction_amt(value):
    if value < lower_bound:
        return 'Muy bajo'
    elif lower_bound <= value < Q1:
        return 'Bajo'
    elif Q1 <= value < Q3:
        return 'Medio'
    elif Q3 <= value < upper_bound:
        return 'Alto'
    else:
        return 'Muy alto'

# Aplicar la función de clasificación a la columna
X['TransactionAmt_Range'] = X['TransactionAmt'].apply(classify_transaction_amt)


# Mostrar algunos resultados
print(X['TransactionAmt_Range'].value_counts())


TransactionAmt_Range
Medio       222081
Bajo         96130
Alto         69619
Muy alto     44838
Name: count, dtype: int64


### Eliminamos la columna 'TransactionAmt' por la conclusiones obtenidas en EDA

In [11]:
X.drop('TransactionAmt', axis=1, inplace=True)

### Juntar addr1 y addr2

- addr1 representa una región dentro de un país, mientras que addr2 corresponde al código de país. Combinar estas variables podría capturar de manera más efectiva la relación geográfica entre regiones y países en el modelo.

In [12]:
X['addr1'] = X['addr1'].astype('int').astype('str')
X['addr2'] = X['addr2'].astype('int').astype('str')

# Concatenar addr1 y addr2 en una nueva columna addr_combined usando +
X['addr_combined'] = X['addr1'] + '_' + X['addr2']

In [13]:
X.columns

Index(['TransactionDT', 'ProductCD', 'addr1', 'addr2', 'P_emaildomain',
       'TransactionAmt_Range', 'addr_combined'],
      dtype='object')

In [14]:
## Eliminamos addr1 y addr2 de X
X.drop(['addr1', 'addr2'],axis=1, inplace=True)

### Codificación de variables categóricas

In [15]:
X.dtypes

TransactionDT            int64
ProductCD               object
P_emaildomain           object
TransactionAmt_Range    object
addr_combined           object
dtype: object

In [16]:
# Identificar tipos de datos
data_types = X.dtypes

# Filtrar y contar variables numéricas y categóricas
num_vars = data_types[data_types != 'object']  # Variables numéricas
cat_vars = data_types[data_types == 'object']  # Variables categóricas
num_vars_names = num_vars.index.tolist()
cat_vars_names = cat_vars.index.tolist()

In [17]:
for columna in cat_vars_names :
    print(f"Columna: {columna}, el número de variables es: {X[columna].nunique()}")

Columna: ProductCD, el número de variables es: 4
Columna: P_emaildomain, el número de variables es: 59
Columna: TransactionAmt_Range, el número de variables es: 4
Columna: addr_combined, el número de variables es: 424


Debido a que cada columna tiene diferentes tipos de categorías, aplicaremos según sea adecuada a cada cado.
- ProductCD (4 categorías):
    - One-Hot Encoding: Como ProductCD tiene un número pequeño y fijo de categorías (4 en total), el one-hot encoding es una opción adecuada. 

- TransactionAmt_Range (4 categorías):
    - Dado que TransactionAmt_Range tiene un orden natural o ordinal con 4 categorías, el ordinal encoding es apropiado. Asigna valores numéricos secuenciales a cada categoría.
- P_emaildomain (56 categorías) y  addr_combined:
    - Hashing Trick o Binary Encoding:  Dado que P_emaildomain tiene muchas categorías (56), el one-hot encoding puede generar demasiadas columnas y aumentar la complejidad. binary encoding podrían ser más eficientes.


In [18]:
p.head()

NameError: name 'p' is not defined

In [ ]:

# Codificación One-Hot para ProductCD
encoder_productcd = OneHotEncoder()
X_productcd_encoded = encoder_productcd.fit_transform(X[['ProductCD']])


# Convertir la matriz dispersa a DataFrame y verificar su forma
X_productcd_encoded_df = pd.DataFrame(X_productcd_encoded.toarray(), 
                                      columns=encoder_productcd.get_feature_names_out(['ProductCD']),
                                      index=X.index)

X.drop(['ProductCD'], axis=1,inplace=True)
X  = pd.concat([X, X_productcd_encoded_df], axis=1)

# Codificación Ordinal para TransactionAmt_Range
encoder_transactionamt_range = OrdinalEncoder()
X['TransactionAmt_Range'] = encoder_transactionamt_range.fit_transform(X[['TransactionAmt_Range']])

# Codificación BinaryEncoder para 'P_emaildomain'
encoder_pemaildomain = ce.BinaryEncoder(cols=['P_emaildomain'])
X_encoded_pemaildomain = encoder_pemaildomain.fit_transform(X['P_emaildomain'])

# Eliminar la columna original 'P_emaildomain' y concatenar las columnas codificadas
X = pd.concat([X.drop(['P_emaildomain'], axis=1), X_encoded_pemaildomain], axis=1)



# Codificación BinaryEncoder para 'addr_combined'
encoder_addr_combined = ce.BinaryEncoder(cols=['addr_combined'])
X_encoded_addr_combined = encoder_addr_combined.fit_transform(X['addr_combined'])

# Eliminar la columna original 'addr_combined' y concatenar las columnas codificadas
X = pd.concat([X.drop(['addr_combined'], axis=1), X_encoded_addr_combined], axis=1)



print( X.shape)

(432668, 21)


In [ ]:
X.columns

Index(['TransactionDT', 'TransactionAmt_Range', 'ProductCD_C', 'ProductCD_H',
       'ProductCD_R', 'ProductCD_W', 'P_emaildomain_0', 'P_emaildomain_1',
       'P_emaildomain_2', 'P_emaildomain_3', 'P_emaildomain_4',
       'P_emaildomain_5', 'addr_combined_0', 'addr_combined_1',
       'addr_combined_2', 'addr_combined_3', 'addr_combined_4',
       'addr_combined_5', 'addr_combined_6', 'addr_combined_7',
       'addr_combined_8'],
      dtype='object')

## Normalización y estandarización de las variables.

- Basados en el anális EDA la columna 'TransactionDT' tiene una distribución casi uniforme por lo tanto la normalización Min-Max ajusta los valores de una característica a un rango específico, típicamente entre 0 y 1. Es útil cuando deseas mantener la distribución de los datos pero ajustarla a una escala uniforme.

In [ ]:
scaler = MinMaxScaler()

# Ajustar y transformar los datos de TransactionDT
X['TransactionDT_scaled'] = scaler.fit_transform(X[['TransactionDT']])
X.drop('TransactionDT',axis=1,inplace=True)

## Conclusión

En este proyecto de Data Science, el proceso de limpieza y preprocesamiento de datos ha sido fundamental para preparar nuestros datos antes de aplicar modelos de aprendizaje automático. A continuación, se detallan las principales etapas y decisiones tomadas durante este proceso:

1. **Selección Estratégica del Dataset**: Se cargó el dataset completo y se seleccionó estratégicamente una porción del 10%, utilizando muestreo estratificado para asegurar representatividad y evitar sesgos en nuestros modelos.

2. **Selección de Características Relevantes**: Se llevó a cabo la selección de las columnas más relevantes para nuestro análisis y modelos, descartando aquellas que no contribuían significativamente a la predicción del target.

3. **Manejo de Valores Nulos**: Se eliminaron aquellas columnas que contenían un alto porcentaje (mayor al 80%) de valores nulos, así como las filas con un porcentaje bajo de valores nulos para garantizar la integridad de los datos restantes.

4. **Separación de Variables**: Se realizó una clara separación entre las variables características (`X`) y el target (`y`), asegurando que estuvieran correctamente definidas para el entrenamiento de los modelos.

5. **Ingeniería de Características**: Basados en un análisis exploratorio de datos (EDA), se reemplazó la columna `TransactionAmt` por `TransactionAmt_Range`, una variable categórica que agrupa los valores en rangos como "muy bajo", "bajo", "medio", "alto" y "muy alto". Esta transformación facilita el manejo de variables con amplios rangos de valores, evitando posibles complicaciones durante el entrenamiento de los modelos.

6. **Combinación de Variables**: Se combinaron las columnas `addr1` y `addr2` en una sola columna (`addr_combined`), reduciendo así la dimensionalidad del dataset sin perder información relevante.

7. **Codificación de Variables Categóricas**: Se aplicaron técnicas adecuadas de codificación a las variables categóricas según su naturaleza, como codificación one-hot y binary encoding, para prepararlas para su uso en los modelos de aprendizaje automático.

8. **Normalización de Variables Numéricas**: Se normalizaron las variables numéricas para asegurar que todas estuvieran en la misma escala, lo cual es crucial para modelos que se basan en la distancia o magnitud de los atributos.

En resumen, el proceso de limpieza y preprocesamiento de datos realizado ha permitido transformar el dataset inicial en un formato apto y optimizado para la construcción de modelos predictivos. Estas etapas son fundamentales para asegurar la calidad de los resultados obtenidos y facilitar la interpretación y aplicación de los modelos en la práctica.


In [ ]:
# Guardar X y y en archivos CSV

dir_data_processed = '../data/processed'
ruta_archivo_X = os.path.join(dir_data_processed, 'datos_procesados.csv')
X.to_csv(ruta_archivo_X, index=False, encoding='utf-8')

ruta_archivo_y = os.path.join(dir_data_processed, 'target.csv')
y.to_csv(ruta_archivo_y, index=False, encoding='utf-8')

